#  Incident Root Cause Analysis 

Incident Reports in ITOps usually states the symptoms. Identifying the root cause of the symptom quickly is a key determinant to reducing resolution times and improving user satisfaction.

In [1]:
#Install all related packages. If you find additional packages missing, please follow the same technique.
#If you are not using anaconda, then use pip to install the same packages

import sys
!conda install --yes --prefix {sys.prefix} pandas tensorflow scikit-learn

Solving environment: done

# All requested packages already installed.



# 02.04. Preprocessing Incident Data

### Loading the Dataset

In [5]:
import pandas as pd
import os
import tensorflow as tf

#Load the data file into a Pandas Dataframe
symptom_data = pd.read_csv("/Users/rubenpereira/Desktop/Ex_Files_Applied_AI_IT_Ops/Exercise Files/root_cause_analysis.csv")

#Explore the data loaded
print(symptom_data.dtypes)
symptom_data.head()

ID              int64
CPU_LOAD        int64
MEMORY_LOAD     int64
DELAY           int64
ERROR_1000      int64
ERROR_1001      int64
ERROR_1002      int64
ERROR_1003      int64
ROOT_CAUSE     object
dtype: object


,ID,CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY
1,2,0,0,0,0,0,0,1,MEMORY
2,3,0,1,1,0,0,1,1,MEMORY
3,4,0,1,0,1,1,0,1,MEMORY
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


### Convert  data

Input data needs to be converted to formats that can be consumed by ML algorithms

In [3]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
symptom_data['ROOT_CAUSE'] = label_encoder.fit_transform(
                                symptom_data['ROOT_CAUSE'])

#Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

#Extract the feature variables (X)
X_train = np_symptom[:,1:8]

#Extract the target variable (Y), convert to one-hot-encoding
Y_train=np_symptom[:,8]
Y_train = tf.keras.utils.to_categorical(Y_train,3)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (1000, 7)
Shape of target variable : (1000, 3)


## 02.05. Building the Model with Keras

In [4]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup Training Parameters
EPOCHS=20
BATCH_SIZE=100
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model, using Adam optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 0s 130us/sample - loss: 1.0134 - acc: 0.6750 - val_loss: 0.9273 - val_acc: 0.7600
Epoch 2/20
800/800 [==============================] - 0s 16us/sample - loss: 0.8485 - acc: 0.8300 - val_loss: 0.7861 - val_acc: 0.7800
Epoch 3/20
800/800 [==============================] - 0s 19us/sample - loss: 0.7036 - acc: 0.8288 - val_loss: 0.6632 - val_acc: 0.7800
Epoch 4/20
800/800 [==============================] - 0s 15us/sample - loss: 0.5782 - acc: 0.8400 - val_loss: 0.5800 - val_acc: 0.8000
Epoch 5/20
800/800 [==============================] - 0s 20us/sample - loss: 0.4985 - acc: 0.8375 - val_loss: 0.5486 - val_acc: 0.7900
Epoch 6/20
800/800 [==============================] - 0s 16us/sample - loss: 0.4587 - acc: 0.8338 - val_loss: 0.5335 - val_acc: 0.7900
Epoch 7/20
800/800 [=====

## 02.06. Predicting Root Causes

In [17]:
import numpy as np

#Pass individual flags to Predict the root cause
CPU_LOAD=1
MEMORY_LOAD=0
DELAY=0
ERROR_1000=0
ERROR_1001=1
ERROR_1002=1
ERROR_1003=0

prediction=model.predict_classes(
    np.array([[CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]]))

print(label_encoder.inverse_transform(prediction))

['DATABASE_ISSUE']


In [22]:
#Predicting as a Batch
print(label_encoder.inverse_transform(
        model.predict_classes(np.array([[1,0,0,0,1,1,0],
                                [0,1,1,1,0,0,0],
                                [1,1,0,1,1,0,1],
                                [0,0,0,0,0,1,0],
                                [1,0,1,0,1,1,1]]))))

['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']


In [25]:
arr = np.array([[CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]])
arr.shape

(1, 7)

In [27]:
arr1 = np.array([[1,0,0,0,1,1,0],
                                [0,1,1,1,0,0,0],
                                [1,1,0,1,1,0,1],
                                [0,0,0,0,0,1,0],
                                [1,0,1,0,1,1,1]])
arr1.shape

(5, 7)